# Welcome to the E-Health Workshop

You should have a group, an aws account and even a raspberry pi by now. Just follow along and enjoy! Call us anytime, we're here to help.

## Soundcheck

This section is just to check if your workshop stack is healthy and ready to roll!


Let's see if AWS CLI is working ...

In [ ]:
%%bash
aws ec2 describe-regions --output text

The API call bellow lists the availability zones for your account. Just to make sure AWS SDK for Python, boto3, is setup and authenticating correctly.

In [ ]:
import boto3

ec2 = boto3.client('ec2')

# Retrieves all regions/endpoints that work with EC2
response = ec2.describe_regions()
print('Regions:', response['Regions'])

# Retrieves availability zones only for region of the ec2 object
response = ec2.describe_availability_zones()
print('Availability Zones:', response['AvailabilityZones'])


A correct response would look like this:

![title](img/setup-ok.png)

Do not proceed if you have errors in the cells above, as accessing AWS is essential to this workshop. Call us if you see any error or need some help.

That's it, you're all set, enjoy!

# AWS IoT Basics


Create a Thing

In [ ]:
! aws iot create-thing --thing-name NotebookThing

Create a Certificate

In [ ]:
certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile ./certificate.pem \
  --public-key-outfile ./public-key.pem \
  --private-key-outfile ./private-key.pem \
  --query certificateArn \
  --output text
certificateArn = certificateArn.s

In [ ]:
certificateArn

Download CA


In [ ]:
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O ca.pem

In [ ]:
%%bash
ls -liah *.pem

In [ ]:
policyArn = ! aws iot create-policy \
    --policy-name NotebooksPolicy \
    --policy-document '{"Version": "2012-10-17","Statement": [{"Effect": "Allow","Action": "iot:*","Resource": "*"}]}' \
    --query policyArn \
    --output text 

In [ ]:
policyArn

Attach certificate to thing and policy

In [ ]:
! aws iot attach-policy \
    --policy-name NotebooksPolicy \
    --target {certificateArn}

In [ ]:
! aws iot attach-thing-principal \
  --thing-name NotebookThing \
  --principal {certificateArn}

Find out your endpoint

In [ ]:
endpointAddress = !aws iot describe-endpoint --query endpointAddress --output=text
endpointAddress = endpointAddress.s


In [ ]:
endpointAddress

Sending a message

In [ ]:
! aws iot-data publish --topic "notebook-topic" --payload "{'hello':'from AWS CLI'}" 

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

mqtt = AWSIoTMQTTClient("NotebookCient")
mqtt.configureEndpoint('a3p88ooj9y6ruv.iot.us-east-1.amazonaws.com', 8883)
mqtt.configureCredentials("ca.pem", "private-key.pem", "certificate.pem")
mqtt.configureConnectDisconnectTimeout(10)
mqtt

Send a message

In [ ]:
mqtt.connect()

In [ ]:
mqtt.publish("notebook-topic", '{"helo":"from Jupyter" }' , 0)

In [ ]:
mqtt.disconnect()